In [72]:
import discretisedfield as df
import micromagneticmodel as mm

import oommfc as oc
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib.patheffects as patheffects
%matplotlib qt5

In [24]:
read_field = df.Field.fromfile("magnetization.ovf")
read_field.mesh

Mesh(Region(p1=(0.0, 0.0, 0.0), p2=(1.49504e-05, 1.49504e-05, 2e-08)), n=(2048, 2048, 2), attributes: (unit: m, fourierspace: False, isplane: False))

In [54]:
zlims = (read_field.mesh.region.p1[2], read_field.mesh.region.p2[2])
magnet_w = (
    read_field.mesh.region.p2[0] - read_field.mesh.region.p1[0]
) / 8  # assumes symmetric
magnet_x = 3
magnet_y = 3
rm_margin = 0.15 * magnet_w
subregion = df.Region(
    p1=(magnet_w * magnet_x + rm_margin, magnet_w * magnet_y + rm_margin, zlims[0]),
    p2=(magnet_w * (magnet_x + 1) - rm_margin, magnet_w * (magnet_y + 1) - rm_margin, zlims[1]),
)

submesh = df.Mesh(region=subregion, cell=(1, 1, 1))

Field(Mesh(Region(p1=(5.8837999999999995e-06, 5.8837999999999995e-06, 0.0), p2=(7.197800000000001e-06, 7.197800000000001e-06, 2e-08)), n=(180, 180, 2), attributes: (unit: m, fourierspace: False, isplane: False)), dim=3, components: (x, y, z), units=1)

In [56]:
zlims = (read_field.mesh.region.p1[2], read_field.mesh.region.p2[2])
magnet_w = (
    read_field.mesh.region.p2[0] - read_field.mesh.region.p1[0]
) / 8  # assumes symmetric
magnet_x = 3
magnet_y = 3
rm_margin = 0.15 * magnet_w
subregion = df.Region(
    p1=(magnet_w * magnet_x + rm_margin, magnet_w * magnet_y + rm_margin, zlims[0]),
    p2=(magnet_w * (magnet_x + 1) - rm_margin, magnet_w * (magnet_y + 1) - rm_margin, zlims[1]),
)

submesh = df.Mesh(region=subregion, cell=(1, 1, 1))
fig, ax = plt.subplots()
read_field[subregion].plane("z", n=(70, 70)).mpl(
    ax=ax,
    multiplier=1e-6,
    vector_kw={
        "colorbar_label": "z-component of magnetic vector",
        "scale": 50,
    },
    scalar_kw={
        "clim": (-0.2, 0.2)
    }
)
ax.axis("off")
plt.show()

In [103]:
def add_scalebar(ax: plt.Axes): 
    scalebar_kwargs = {'size': 0.5, 'label': '500nm', 'loc': 8, 'frameon': False, 'color': 'black', 'size_vertical': 0.02, 'label_top': False, 'fontproperties': fm.FontProperties(size=18)}
    scalebar = AnchoredSizeBar(transform=ax.transData, **scalebar_kwargs)
    # Denne legger til et svart omriss rundt scalebar teksten, for å gjøre den lettere å lese
    scalebar.txt_label._text.set_path_effects([patheffects.withStroke(linewidth=2, foreground='black', capstyle="round")])
    ax.add_artist(scalebar)

In [106]:
zlims = (read_field.mesh.region.p1[2], read_field.mesh.region.p2[2])
magnet_w = (
    read_field.mesh.region.p2[0] - read_field.mesh.region.p1[0]
) / 8  # assumes symmetric
magnet_x = 6
magnet_y = 6
rm_margin = 0.15 * magnet_w
subregion = df.Region(
    p1=(magnet_w * magnet_x + rm_margin, magnet_w * magnet_y + rm_margin, zlims[0]),
    p2=(magnet_w * (magnet_x + 1) - rm_margin, magnet_w * (magnet_y + 1) - rm_margin, zlims[1]),
)

submesh = df.Mesh(region=subregion, cell=(1, 1, 1))
fig, ax = plt.subplots()
read_field[subregion].plane("z").mpl.lightness(
    ax=ax, cmap="turbo", lightness_field=read_field[subregion].z, colorwheel_args={"loc": 1}
)
read_field[subregion].plane(z=10e-9, n=(30, 30)).mpl.vector(
    ax=ax, scale=30, colorbar=False, use_color=False
)

add_scalebar(ax)
ax.axis("off")
fig.savefig("single_horseshoe_sim.png", bbox_inches="tight", pad_inches=0)
